In [1]:
import praw
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [11]:
!pip3 install numpy

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
reddit_read_only = praw.Reddit(client_id="xnsIxViGuyQrTKs7VnnYCQ",  # your client id
                               client_secret="LuRt_v69n1GxxADiaLssD25fWOkuyw",  # your client secret
                               user_agent="safespace",
                               redirect_uri="http://localhost:8080")

In [3]:
subredditList = ["traumatoolbox",
"rapecounseling",
"7CupsofTea",
"addiction",
"ADHD",
"Advice",
"affirmations",
"afterthesilence",
"Agoraphobia",
"alcoholism",
"Anger",
"Antipsychiatry",
"Anxiety",
"AskDocs",
"BipolarReddit",
"BPD",
"BodyAcceptance",
"bulimia",
"CompulsiveSkinPicking",
"dbtselfhelp",
"depression_help",
"disability",
"domesticviolence",
"EatingDisorders",
"EOOD",
"ForeverAlone",
"getting_over_it",
"HealthAnxiety",
"helpmecope",
"mentalhealth",
"mentalillness",
"medical",
"MMFB",
"Needafriend",
"OCD",
"offmychest",
"Psychiatry",
"ptsd",
"reasonstolive",
"rehabtherapy",
"sad",
"secondary_survivors",
"selfharm",
"selfhelp",
"siblingsupport",
"SMARTRecovery",
"socialanxiety",
"socialskills",
"stopsmoking",
"SuicideWatch",
"uniqueminds",
"whatsbotheringyou",
"relationship_advice",
"howtonotgiveafuck",
"selfimprovement",
"personalfinance",
"confidence",
"love",
"lonely",
"GriefSupport",
"LongDistance",
"Divorce",
"AskWomen",
"AskMen",
"nus",
"SGExams",
]

In [5]:
data = {"subreddit": [], "content":[]}

for i in subredditList:
    print(i)
    try:
        subreddit = reddit_read_only.subreddit(i)
        data["subreddit"].append(i)
        data["content"].append(subreddit.description)
        for post in subreddit.hot(limit=500):
                data["subreddit"].join(i)
                string = ""
                string.join(post.title)
                string.join(" ")
                string.join(post.selftext)
                data["content"].join(string)
    except Exception:
        continue




traumatoolbox
rapecounseling
7CupsofTea
addiction
ADHD
Advice
affirmations
afterthesilence
Agoraphobia
alcoholism
Anger
Antipsychiatry
Anxiety
AskDocs
BipolarReddit
BPD
BodyAcceptance
bulimia
CompulsiveSkinPicking
dbtselfhelp
depression_help
disability
domesticviolence
EatingDisorders
EOOD
ForeverAlone
getting_over_it
HealthAnxiety
helpmecope
mentalhealth
mentalillness
medical
MMFB
Needafriend
OCD
offmychest
Psychiatry
ptsd
reasonstolive
rehabtherapy
sad
secondary_survivors
selfharm
selfhelp
siblingsupport
SMARTRecovery
socialanxiety
socialskills
stopsmoking
SuicideWatch
uniqueminds
whatsbotheringyou
relationship_advice
howtonotgiveafuck
selfimprovement
personalfinance
confidence
love
lonely
GriefSupport
LongDistance
Divorce
AskWomen
AskMen
nus
SGExams


In [6]:
data = pd.DataFrame(data)

In [7]:
data

,subreddit,content
0,traumatoolbox,r/Traumatoolbox is a place to seek or share co...
1,rapecounseling,**RapeCounseling is dedicated to providing emo...
2,7CupsofTea,[](http://www.7cups.com/chat/getChatFind.php?i...
3,addiction,###Welcome to /r/Addiction.\n\n> ##[What is A...
4,ADHD,##### [](#derp)\n*****************************...
...,...,...
61,Divorce,"Information for divorced parents, children, an..."
62,AskWomen,###### [Don't forget to upvote good questions!...
63,AskMen,> ###Community Rules:\n\n\n\n> **DON'T BE AN A...
64,nus,Welcome to the **[National University of Singa...


In [8]:
data.to_csv("my_community_new_data_500.csv")


In [9]:
len(subredditList)

66

In [10]:
read_data = pd.read_csv("my_community_new_data_500.csv")
read_data

,Unnamed: 0,subreddit,content
0,0,traumatoolbox,r/Traumatoolbox is a place to seek or share co...
1,1,rapecounseling,**RapeCounseling is dedicated to providing emo...
2,2,7CupsofTea,[](http://www.7cups.com/chat/getChatFind.php?i...
3,3,addiction,###Welcome to /r/Addiction.\n\n> ##[What is A...
4,4,ADHD,##### [](#derp)\n*****************************...
...,...,...,...
61,61,Divorce,"Information for divorced parents, children, an..."
62,62,AskWomen,###### [Don't forget to upvote good questions!...
63,63,AskMen,> ###Community Rules:\n\n\n\n> **DON'T BE AN A...
64,64,nus,Welcome to the **[National University of Singa...


In [11]:
le = preprocessing.LabelEncoder()
y_index = le.fit_transform(read_data.subreddit.values)
read_data['class'] = y_index
read_data['subreddit'].to_csv("my_community_new_labels_500.csv")

In [77]:
data

,subreddit,content
0,traumatoolbox,r/Traumatoolbox is a place to seek or share co...
1,rapecounseling,**RapeCounseling is dedicated to providing emo...
2,7CupsofTea,[](http://www.7cups.com/chat/getChatFind.php?i...
3,addiction,###Welcome to /r/Addiction.\n\n> ##[What is A...
4,ADHD,##### [](#derp)\n*****************************...
...,...,...
61,Divorce,"Information for divorced parents, children, an..."
62,AskWomen,###### [Don't forget to upvote good questions!...
63,AskMen,> ###Community Rules:\n\n\n\n> **DON'T BE AN A...
64,nus,Welcome to the **[National University of Singa...


In [12]:
read_data

,Unnamed: 0,subreddit,content,class
0,0,traumatoolbox,r/Traumatoolbox is a place to seek or share co...,63
1,1,rapecounseling,**RapeCounseling is dedicated to providing emo...,50
2,2,7CupsofTea,[](http://www.7cups.com/chat/getChatFind.php?i...,0
3,3,addiction,###Welcome to /r/Addiction.\n\n> ##[What is A...,28
4,4,ADHD,##### [](#derp)\n*****************************...,1
...,...,...,...,...
61,61,Divorce,"Information for divorced parents, children, an...",14
62,62,AskWomen,###### [Don't forget to upvote good questions!...,9
63,63,AskMen,> ###Community Rules:\n\n\n\n> **DON'T BE AN A...,8
64,64,nus,Welcome to the **[National University of Singa...,46


In [13]:
read_data['content'] = read_data['content'].astype(str)

In [14]:
stemmer = PorterStemmer()
words = stopwords.words('english')

read_data['cleaned'] = read_data['content'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i.lower() not in words and len(i) > 1]).lower())

In [15]:
read_data

,Unnamed: 0,subreddit,content,class,cleaned
0,0,traumatoolbox,r/Traumatoolbox is a place to seek or share co...,63,traumatoolbox place seek share cope strategi r...
1,1,rapecounseling,**RapeCounseling is dedicated to providing emo...,50,rapecounsel dedic provid emot support experien...
2,2,7CupsofTea,[](http://www.7cups.com/chat/getChatFind.php?i...,0,http www cup com chat getchatfind php welcom c...
3,3,addiction,###Welcome to /r/Addiction.\n\n> ##[What is A...,28,welcom addict addict icon help term addict use...
4,4,ADHD,##### [](#derp)\n*****************************...,1,derp see discordwidget info help titleproblem ...
...,...,...,...,...,...
61,61,Divorce,"Information for divorced parents, children, an...",14,inform divorc parent children friend relev ple...
62,62,AskWomen,###### [Don't forget to upvote good questions!...,9,forget upvot good question http www reddit com...
63,63,AskMen,> ###Community Rules:\n\n\n\n> **DON'T BE AN A...,8,commun rule asshol unsur rule appli post comme...
64,64,nus,Welcome to the **[National University of Singa...,46,welcom nation univers singapor http www nu edu...


In [16]:
x_train, x_test, y_train, y_test = train_test_split(read_data['cleaned'], read_data['class'], test_size = 0.2)

In [17]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=6000)),
                     ('clf', RandomForestClassifier(n_estimators= 200, min_samples_split= 20))])

In [18]:
model = pipeline.fit(x_train, y_train)

In [19]:
print("accuracy score: " + str(model.score(x_test, y_test)))

accuracy score: 0.0


In [20]:
model = pipeline.fit(read_data['cleaned'], read_data['class'])

In [21]:
from joblib import dump
dump(model, 'mycommunity_new_redditModel500.joblib', compress =3)

['mycommunity_new_redditModel500.joblib']

In [22]:
test = """

 I have been having these recurring thoughts lately, and since they are more frequent than ever, I know these are emotional baggages that I have to release before I move on.

Having just entered my 30s, sometimes I look back at the last decade and wonder how things could have been had I done them differently, especially during my uni years. 

Many times, I wish I could have done way more back when I was in NUS. I felt like I mugged too much, closed myself from several opportunities, and did not spend a lot of time doing the things that I love to do (and I wish I did).

These things include but not limited to signing up for more gigs and making more music (since I have always loved music from Day 1), joining more student clubs and societies in NUS, and making more friends in and around the campus, even though those were arguably some of the most carefree times I have ever had in my life. 

Being a somewhat shy individual back in the day, I did not have the self-confidence to do many of these things out of fear of what others were going to say. It was only after I graduated that I realised it was already too late. 

Now that I am very confident of who I am as a person and know myself a lot better over the years, while acknowledging that it is certainly impossible to turn back time, I sometimes still wish I was a lot more outgoing and open as a person 8-12 years ago. Maybe things would have been rather different back then.

Nevertheless, thank you all if you read this till the very end. 

P.S. I am good and healthy by the way! I just want to express a few regrets that I kept to myself for years, that is all.

"""
y_predprob = model.predict_proba([test])
pred_To_Class_map = pd.DataFrame(y_predprob, columns=le.classes_)
pred_To_Class_map = pred_To_Class_map.transpose()
pred_To_Class_map = pred_To_Class_map.rename(columns = {0 : "prob"})
pred_To_Class_map = pred_To_Class_map.sort_values(by = "prob", ascending= False)
recommendationList = [i for i in pred_To_Class_map.head(5).transpose()]
recommendationList



['socialanxiety',
 'Agoraphobia',
 'afterthesilence',
 'selfhelp',
 'CompulsiveSkinPicking']

In [ ]:
# reddit_read_only = praw.Reddit(client_id="xnsIxViGuyQrTKs7VnnYCQ",  # your client id
#                                client_secret="LuRt_v69n1GxxADiaLssD25fWOkuyw",  # your client secret
#                                user_agent="safespace",
#                                redirect_uri="http://localhost:8080")

urlAndInfoDict = {}

for i in recommendationList:
    subreddit = reddit_read_only.subreddit(i)
    urlAndInfoDict[subreddit.title] = [subreddit.public_description, subreddit.url, subreddit.subscribers]

urlAndInfoDict


In [ ]:
redditData = {"subreddit":[] ,"public_description": [], "url":[], "subscribers" : []}
for i in subredditList:
    subreddit = reddit_read_only.subreddit(i)
    redditData["subreddit"].append(i)
    redditData["public_description"].append(subreddit.public_description)
    redditData["url"].append(subreddit.url)
    redditData["subscribers"].append(subreddit.subscribers)

redditData

In [ ]:
redditData = pd.DataFrame(redditData)
redditData.to_csv("redditData.csv")

In [ ]:
from joblib import load

test = """
I’m 23 year old and a 3rd year student in an university. I grew up in a single income family. My dad made the choice to womanize and abandoned 2 young kids then with my mom. He tells everyone else he have no money but he posts on social media pictures of him traveling all over the world while eating food. He refuses to pay anything for my younger sibling and my education, living expenses etc. He used to pay a bit like $1k every 6 months when we are much younger like in primary school. He cut us off officially financially when I’m 18 years old telling me straight in the face that we are “ a burden to him “ while he is smoking. He barely fed us, clothe us but expect us to give him an X amount in future so he will sue us for the maintenance thing. 

 Please tell me, is it not parent’s responsibility to bring kids up since they are the ones who chose to have unprotected sex? 

My mother is a strong woman in everyone’s eyes. She single-handedly brought me and my younger sibling up. She refuse to believe it but the divorce tore her up and the lack of financial support from my dad. She worked hard. Working endlessly. To support my younger sibling and me. Whenever she’s sad or being emotional, she will use emotional/mental abuse like guilt tripping, manipulation, gaslighting, etc. This is her way of being human after being strong for so long. I’m often the target as I’m not academically inclined since young as compared to my younger sibling.  At times it really gets into me. I would cry and beat/bite/scratch myself thinking I’m not good enough. With all the negative thoughts. Basically self harm, i know. I went to counseling once but stop as i was worried it will affect my future career prospects. Like will the employer be judgmental when they found out? At times i really want to move out but it’s not sustainable in Singapore as prices is so crazy expensive. So i can only endure and suck it up. That’s my coping mechanism for a long time. 

Before anyone bash me up saying i should suck it up as everyone have a coping mechanism and i should understand, what about me? Who will think of me?"""


df_y = pd.read_csv('labels_500.csv', index_col=0)
model = load('redditModel500.joblib')
redditData = pd.read_csv("redditData.csv")
redditDict = {}
for i in redditData.values:
    redditDict[i[1]] = {"public_description": i[2], "url": i[3],
                                        "subscribers": i[4]}

le = preprocessing.LabelEncoder()
le.fit(df_y['subreddit'].unique())

LabelEncoder()

In [ ]:
y_predprob = model.predict_proba([test])
pred_To_Class_map = pd.DataFrame(y_predprob, columns=le.classes_)
pred_To_Class_map = pred_To_Class_map.transpose()
pred_To_Class_map = pred_To_Class_map.rename(columns={0: "prob"})
pred_To_Class_map = pred_To_Class_map.sort_values(by="prob", ascending=False)
recommendationList = [i for i in pred_To_Class_map.head(5).transpose()]
urlAndInfoDict = {}

for i in recommendationList:
    urlAndInfoDict[i] = {"public_description": redditDict[i]["public_description"], "url": redditDict[i]["url"],
                                        "subscribers": redditDict[i]["subscribers"]}

In [ ]:
urlAndInfoDict

{'selfharm': {'public_description': 'A subreddit for self-harmers to relate to each other, ask questions, and build up a community.',
  'url': '/r/selfharm/',
  'subscribers': 97542},
 'ForeverAlone': {'public_description': "A subreddit for Forever Alone folks.\n\nOfficial Discord server: https://discord.com/invite/TvDz9jB\n\nIf you're hurting: /r/MMFB/comments/m48gq/updated_list_of_hotline_s_for_those_who_need_them/), or you can call\n\n1-800-273-TALK (8255) or 1-866-APPELLE (if in Quebec). You can also check\n\n/r/SuicideWatch",
  'url': '/r/ForeverAlone/',
  'subscribers': 170018},
 'offmychest': {'public_description': "A mutually supportive community where deeply emotional things you can't tell people you know can be told. \n\nWhether it's long-standing baggage, happy thoughts, or recent trauma, posting it here may provide some relief. We'll listen, and if you want, we'll talk. \n\nWe aim to keep this a safe space.",
  'url': '/r/offmychest/',
  'subscribers': 2682256},
 'mentalhea

In [ ]:

redditDict

{'traumatoolbox': {'public_description': 'A place to seek or share coping strategies, resources, art, music, videos, and other survivor tools.',
  'url': '/r/traumatoolbox/',
  'subscribers': 14786},
 'rapecounseling': {'public_description': 'RapeCounseling is a Reddit forum dedicated to providing an open forum for survivors and victims of sexualized violence across the spectrum.',
  'url': '/r/rapecounseling/',
  'subscribers': 23540},
 '7CupsofTea': {'public_description': 'Whether you’re feeling stressed, or just want to get something off your chest, connecting with someone through www.7cups.com is a convenient way to feel better. We connect you to volunteers trained in active listening, a method of emotional support known to relieve stress. Listeners are non-judgmental and kind. All conversations are free, anonymous, and done via text chat.\nAdditionally you can do many simple exercises on your own to stay relaxed and calm.',
  'url': '/r/7CupsofTea/',
  'subscribers': 5228},
 'addi